<a href="https://colab.research.google.com/github/qatatsha-curtin/ISYS5002/blob/main/Stagging2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import spacy
import re

nlp = spacy.load("en_core_web_sm")

def parse_weather_question(question):
    """
    Extract only locations and number of forecast days from a natural language question.

    Args:
        question (str): User input

    Returns:
        dict: {'locations': [...], 'forecast_days': int}
    """
    doc = nlp(question)

    # 1. Extract locations using GPE entities
    locations = [ent.text for ent in doc.ents if ent.label_ == "GPE"]

    # 2. Contextual fallback: collect consecutive proper nouns at the start if NER fails
    if not locations:
        locations = []
        skip_tokens = {"and", "vs", ","}
        for token in doc:
            text_lower = token.text.lower()
            if text_lower in skip_tokens:
                continue
            if token.pos_ == "PROPN":
                locations.append(token.text.title())
            else:
                break  # Stop at first token that is not a proper noun or skip word

    # 3. Extract numeric forecast_days (e.g., "3 days")
    forecast_days = 5  # default
    numeric_days = re.findall(r'\b(\d+)\s+days?\b', question.lower())
    if numeric_days:
        forecast_days = min(max(int(numeric_days[0]), 1), 5)

    return {
        'locations': locations if locations else None,
        'forecast_days': forecast_days
    }

# Test input
question = "perth and sydney weather and also the rain please for 3 days"
parsed = parse_weather_question(question)
locations = parsed['locations']
forecast_days = parsed['forecast_days']
print(parse_weather_question(question))



{'locations': ['Perth'], 'forecast_days': 3}


In [3]:
import requests

def get_weather_data(location, forecast_days=5):
    """
    Retrieve weather data for a specified location from wttr.in.

    Args:
        location (str): City or location name
        forecast_days (int): Number of days to forecast (1-5)

    Returns:
        dict: Weather data including current conditions and forecast
    """
    # Ensure forecast_days is between 1 and 5
    forecast_days = min(max(forecast_days, 1), 5)

    # Build wttr.in URL for JSON output
    url = f"https://wttr.in/{location}?format=j1"

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise error if request fails
        data = response.json()

        # Extract current condition
        current_condition = data.get("current_condition", [{}])[0]

        # Extract forecast for specified days
        forecast = data.get("weather", [])[:forecast_days]

        # Build simplified result
        result = {
            "location": location,
            "current": current_condition,
            "forecast": forecast
        }

        return result

    except requests.RequestException as e:
        print(f"Error retrieving weather data: {e}")
        return None

# Example usage
# location = "Sydney"
# forecast_days = 3
weather = get_weather_data(locations, forecast_days)
print(weather)


{'location': ['Perth'], 'current': {'FeelsLikeC': '21', 'FeelsLikeF': '70', 'cloudcover': '0', 'humidity': '43', 'localObsDateTime': '2025-10-01 03:14 PM', 'observation_time': '07:14 AM', 'precipInches': '0.0', 'precipMM': '0.0', 'pressure': '1019', 'pressureInches': '30', 'temp_C': '21', 'temp_F': '70', 'uvIndex': '4', 'visibility': '10', 'visibilityMiles': '6', 'weatherCode': '113', 'weatherDesc': [{'value': 'Sunny'}], 'weatherIconUrl': [{'value': ''}], 'winddir16Point': 'SW', 'winddirDegree': '228', 'windspeedKmph': '16', 'windspeedMiles': '10'}, 'forecast': [{'astronomy': [{'moon_illumination': '60', 'moon_phase': 'Waxing Gibbous', 'moonrise': '12:02 PM', 'moonset': '02:05 AM', 'sunrise': '05:55 AM', 'sunset': '06:18 PM'}], 'avgtempC': '16', 'avgtempF': '61', 'date': '2025-10-01', 'hourly': [{'DewPointC': '11', 'DewPointF': '51', 'FeelsLikeC': '15', 'FeelsLikeF': '59', 'HeatIndexC': '15', 'HeatIndexF': '59', 'WindChillC': '15', 'WindChillF': '59', 'WindGustKmph': '11', 'WindGustMil

In [4]:
if not locations:
    print("No locations found in your input.")
else:
    for city in locations:
        weather = get_weather_data(city, forecast_days)
        if weather:
            print(f"\nWeather for {city}:")
            print("Current conditions:", weather['current'])
            print("Forecast:")
            for day in weather['forecast']:
                print(day)


Weather for Perth:
Current conditions: {'FeelsLikeC': '21', 'FeelsLikeF': '70', 'cloudcover': '50', 'humidity': '46', 'localObsDateTime': '2025-10-01 11:05 AM', 'observation_time': '03:05 AM', 'precipInches': '0.0', 'precipMM': '0.0', 'pressure': '1022', 'pressureInches': '30', 'temp_C': '21', 'temp_F': '70', 'uvIndex': '7', 'visibility': '10', 'visibilityMiles': '6', 'weatherCode': '116', 'weatherDesc': [{'value': 'Partly cloudy'}], 'weatherIconUrl': [{'value': ''}], 'winddir16Point': 'SW', 'winddirDegree': '223', 'windspeedKmph': '12', 'windspeedMiles': '7'}
Forecast:
{'astronomy': [{'moon_illumination': '60', 'moon_phase': 'Waxing Gibbous', 'moonrise': '12:02 PM', 'moonset': '02:05 AM', 'sunrise': '05:55 AM', 'sunset': '06:18 PM'}], 'avgtempC': '16', 'avgtempF': '61', 'date': '2025-10-01', 'hourly': [{'DewPointC': '11', 'DewPointF': '51', 'FeelsLikeC': '15', 'FeelsLikeF': '59', 'HeatIndexC': '15', 'HeatIndexF': '59', 'WindChillC': '15', 'WindChillF': '59', 'WindGustKmph': '9', 'Win